In [1]:
import json
import os
import django
import pandas as pd
import sys
sys.path.append('..')
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
os.environ["DJANGO_ALLOW_ASYNC_UNSAFE"] = "true"
os.environ.setdefault('DJANGO_SETTINGS_MODULE', 'SMARTROLL.settings')
django.setup()
from Manage.models import College,Term,Branch,Stream,Semester,Division,Batch,Subject,TimeTable,GPSCoordinates,Classroom,Schedule,Lecture,Link
from Session.models import Attendance,Session
from StakeHolders.models import NotificationSubscriptions,SuperAdmin,Admin,Teacher,Student,Profile
from django.contrib.auth import get_user_model

In [22]:
folder_path = '../student-sheets'
files_with_paths = [os.path.abspath(os.path.join(folder_path, f)) for f in os.listdir(folder_path) if os.path.isfile(os.path.join(folder_path, f))]

In [3]:
def create_student_obj(branch,stream_code,sem,sr_no,enrollment,name):        
    semester_obj = Branch.objects.filter(slug=branch).first().stream_set.filter(stream_code=stream_code).first().semester_set.filter(no=sem).first()
    if semester_obj:
        name = name.rstrip()
        student_obj,student_created = Student.objects.get_or_create(enrollment=enrollment,sr_no=sr_no)
        if student_obj:    
            profile_obj,created = Profile.objects.get_or_create_by_name(name=name,role='student')
            student_obj.profile = profile_obj
            student_obj.save()
            semester_obj.students.add(student_obj)
            return student_obj
        return None

In [10]:
file = open('../student-sheets/3rd_AIML_Students.json')
data = json.load(file)
for division,division_data in data.items():
        for batch,batch_data in division_data.items():
            for student in batch_data:
                student_obj = create_student_obj(branch='332561_1731147390',stream_code='52',sem=3,sr_no=student['sr_no'],enrollment=student['enrollment'],name=student['name'])
                if student_obj:
                    print(f"Student created {student_obj}")
                else:
                    print(f"Student not created {student}")

Student created 230280152001 | AGRAHARI PANKAJ HUKUMACHANDRA
Student created 230280152002 | Alani Manoj Bharatbhai
Student created 230280152003 | Ankush Chunilal Ratnani
Student created 230280152004 | Atharva Jitendra Dholakia
Student created 230280152005 | Balar Maulik Ghanshyambhai
Student created 230280152006 | BAMBHANIYA BRIJESHKUMAR SANABHAI
Student created 230280152007 | Bhammar Krunal Jethurbhai
Student created 230280152008 | Bhatt Priyan Naman
Student created 230280152009 | BUHA VASU MUKESHBHAI
Student created 230280152010 | CHAVDA SHAKTISINH DHIRUBHA
Student created 230280152011 | DASADIYA MESHWA HARIKRUSHNABHAI
Student created 230280152012 | DASALANI JAY NATHALAL
Student created 230280152013 | Desai Anand Kalyanbhai
Student created 230280152014 | DESAI AYUSHI DILIPBHAI
Student created 230280152015 | Dharati Chovatiya
Student created 230280152016 | DHARAV ANTANI
Student created 230280152017 | Dhaval
Student created 230280152018 | DHOLAKIYA MAYUR ASHOKBHAI
Student created 23028

In [4]:
sem_5_students = '/home/manav1011/Documents/SMARTROLL_SSIP_2023/SMARTROLL/notebooks/sheets/5th sem - AIML - 2024 (1).xlsx'

In [8]:
semester = Semester.objects.filter(slug='6367490843084391_1732005086373').first()
df = pd.read_excel(sem_5_students)
for _,row in df.iterrows():    
    sr_no = row.iloc[0]    
    enrollment=row.iloc[1]
    name=row.iloc[2]    
    email=row.iloc[3]
    # create profiles
    profile_obj,created = Profile.objects.get_or_create(name=name,email=email,role='student')
    student,created = Student.objects.get_or_create(profile=profile_obj,sr_no=sr_no,enrollment=enrollment)
    semester.students.add(student)


In [12]:
import random
import string

def generate_password(length=6):
    # Define the characters that can be used in the password (letters and digits)
    characters = string.ascii_letters + string.digits
    
    # Generate a random password by choosing random characters
    password = ''.join(random.choice(characters) for _ in range(length))
    
    return password

In [15]:
students = Student.objects.filter(is_active=False)
for student in students:
    profile=student.profile
    profile.set_password(generate_password())
    profile.save()
